In [71]:
# -*- coding: utf-8 -*-
"""
Module of functions for rebuilding/repairing individual homes and entire
building stocks. Eventually functions for non-residential buildings can be added.

Functions:
home(simulation, human_capital, financial_capital, household, write_story = True, callbacks = None)
stock(simulation, structure_stock, fix_probability, human_capital)

@author: Scott Miles
"""
import sys
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from desaster.config import building_repair_times
from desaster import entities, capitals, request, io, movement, search, rebuild
from simpy import Interrupt
import random
import pandas as pd

In [72]:
simulation = simpy.Environment()

In [73]:
def renew_stock(simulation, structure_stock, fix_percent):
    """Process to rebuild a part or an entire building stock (FilterStore) based
    on available contractors and specified proportion/probability.
    
    structure_stock -- A SimPy FilterStore that contains one or more
    capitals.BuiltCapital(), capitals.Building(), or capitals.Residence() objects 
    that represent vacant structures for purchase.
    """
    random.seed(15)

    structures_list = []  # Empty list to temporarily place FilterStore objects.

    # Remove all structures from the FilterStore; put in a list for processing.
    while True:
        wait_time = 5
        wait = simulation.timeout(wait_time, value='Quit')
        get_structure = structure_stock.get(lambda getStructure:
                                                getStructure.damage_state == 'Moderate' 
                                                or getStructure.damage_state == 'Complete'
                                            )
        loop = yield wait | get_structure
        
        num_fix = int(round(len(structures_list) * fix_percent, 0))
        
        if loop != {wait: 'Quit'}:
            structures_list.append(get_structure)
        else:    
            # Iterate through structures, do processing, put back into the FilterStore
            for put_structure in structures_list[:num_fix]:

                put_structure.damage_state = 'None'
                put_structure.damage_value = 0.0

                structure_stock.put(put_structure)

            print(len(structure_stock.items), len(structures_list), num_fix)
            
            return

In [74]:
file_path = "../inputs/housing_stock_test.csv"
housing_stock_df = pd.read_csv(file_path)

housing_stock = capitals.setHousingStock(simulation, housing_stock_df)

housing_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 New Ave,Mobile Home,100,1,1,700,1920,99999,None
1,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,None
2,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
3,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None
4,104 New Ave,Mobile Home,100,1,1,700,1920,100000,None
5,105 New Ave,Single Family Dwelling,100000,6,5,5000,1920,10000000,None
6,106 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
7,107 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None
8,108 New Ave,Mobile Home,100,1,1,700,1920,100000,Complete
9,109 New Ave,Single Family Dwelling,100000,6,5,5000,1920,10000000,Moderate


In [75]:
fix_percent = 1.0

In [76]:
simulation.process(renew_stock(simulation, housing_stock, fix_percent))

<Process(renew_stock) object at 0x1142aaa58>

In [77]:
simulation.run()

16 8 8


In [78]:
for item in housing_stock.items:
    print(item.address, item.damage_state)

100 New Ave None
101 New Ave None
102 New Ave None
103 New Ave None
104 New Ave None
105 New Ave None
106 New Ave None
107 New Ave None


AttributeError: 'FilterStoreGet' object has no attribute 'address'

In [ ]:
num_fix

In [86]:
housing_stock.items[7].address

'107 New Ave'